# 準備

In [ ]:
!pip install --upgrade xlrd==1.2.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 103 kB 15.9 MB/s 
  Attempting uninstall: xlrd
    Found existing installation: xlrd 1.1.0
    Uninstalling xlrd-1.1.0:
      Successfully uninstalled xlrd-1.1.0


In [ ]:
import urllib.parse
import urllib.request
import json
import os
import pandas as pd
import numpy as np
import xlrd
import openpyxl

In [ ]:
!pip install openpyxl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# 企業活動のCO2

## 単体チェック

EXCELをインポートして整形する関数

In [ ]:
def get_co2(path):

  # Bookをインポートしてシート名をリストに格納
  book = pd.ExcelFile(path)
  sheets = book.sheet_names

  # ファイル名からIDと事業者名を取得
  filename = path.split('/')[6]
  id = filename.split('_')[0]
  company = filename.split('_')[1].replace('.xlsx', '')
  
  # CO2排出量のシートをDataFrameに格納
  df = book.parse(sheets[4])
  
  df = df[['Unnamed: 4','Unnamed: 9','Unnamed: 15','Unnamed: 17','Unnamed: 19']]
  df = df.rename(columns={'Unnamed: 4' : '種類','Unnamed: 9' : '使用量','Unnamed: 15' : '単位','Unnamed: 17' : '排出係数','Unnamed: 19' : 'CO2排出量'})
  df = df.dropna(subset=['種類','使用量','CO2排出量']).reset_index(drop=True)
  
  # 余計なデータを削除
  df = df[df['CO2排出量'] != '－']

  df = df.assign(年次 = '',会社名 = company,ID = id)
  
  for index, row in df.iterrows():
    
    if type(row['CO2排出量']) is str:
      time = '2021年' if index == 0 else '2013年'
      df = df.drop(index)
  
    df['年次'][index] = time

  # 荷役機械のデータが紛れ込むので削除
  df = df[df['排出係数'] != 'LPG']
  df = df[df['排出係数'] != 'ケミカル']
  df = df[df['排出係数'] != '石炭']
  df = df[df['排出係数'] != 'バン頁']
  df = df[df['排出係数'] != '珪砂']
  
  # 改行を削除
  df = df.replace( '\n', '', regex=True)

  # CO2排出量を整数に変換
  df['CO2排出量'] = df['CO2排出量'].astype('int')

  # 排出区分を追加する
  df_type = pd.read_csv('/content/drive/MyDrive/INBOX/企業活動排出区分.csv', encoding = "shift-jis")
  df = pd.merge(df, df_type, on=['種類'] , how='left')
  df = df.fillna({'区分': '電気'})

  # print(df_type)

  return df


パスを指定してテスト

In [ ]:
path = '/content/drive/MyDrive/INBOX/テスト/専01_石塚硝子.xlsx'
df = get_co2(path)
df

/usr/local/lib/python3.7/dist-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


,種類,使用量,単位,排出係数,CO2排出量,年次,会社名,ID,区分
0,揮発油（ガソリン）,600,ℓ,0.0671,1392,2021年,石塚硝子,専01,燃料
1,軽油,17600,ℓ,0.068567,45495,2021年,石塚硝子,専01,燃料
2,液化石油ガス(ＬＰＧ),660,kg,0.059033,1979,2021年,石塚硝子,専01,燃料
3,都市ガス(13A),14923820,Nm3,0.049867,33489052,2021年,石塚硝子,専01,燃料
4,石灰石使用量,6717.9,ｔ,440,2955876,2021年,石塚硝子,専01,工程
5,石灰石使用量,6717.9,ｔ,440,2955876,2021年,石塚硝子,専01,工程
6,ソーダ灰使用量,6998.2,ｔ,415,2904253,2021年,石塚硝子,専01,工程
7,関西電力（ｾﾞﾛｶｰﾎﾞﾝﾒﾆｭｰを除く）,17611880,kWh,0.379,6674902,2021年,石塚硝子,専01,電気
8,関西電力（ｾﾞﾛｶｰﾎﾞﾝﾒﾆｭｰ)以外の電気事業者,12071895,kWh,0.379,4575248,2021年,石塚硝子,専01,電気
9,揮発油（ガソリン）,1000,ℓ,0.0671,2321,2013年,石塚硝子,専01,燃料


合計値を確認しておく

In [ ]:
# 合計値の確認
df_sum = df.groupby(['年次']).agg({'CO2排出量': 'sum'}).reset_index()
df_sum

,年次,CO2排出量
0,2013年,64639195
1,2021年,53604073


## CO2排出明細の作成

ディレクトリ内のファイル一覧を取得する

In [ ]:
def get_dataframe():

  files = os.listdir('drive/MyDrive/INBOX/テスト/')
  
  df_res = pd.DataFrame(index=[], columns=[])
  
  for f in files:
    path = f'/content/drive/MyDrive/INBOX/テスト/{f}'
    df = get_co2(path)
    df_res = pd.concat([df_res, df])

  return df_res

In [ ]:
df_detail = get_dataframe()
df_detail

/usr/local/lib/python3.7/dist-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


,種類,使用量,単位,排出係数,CO2排出量,年次,会社名,ID,区分
0,揮発油（ガソリン）,600,ℓ,0.0671,1392,2021年,石塚硝子,専01,燃料
1,軽油,17600,ℓ,0.068567,45495,2021年,石塚硝子,専01,燃料
2,液化石油ガス(ＬＰＧ),660,kg,0.059033,1979,2021年,石塚硝子,専01,燃料
3,都市ガス(13A),14923820,Nm3,0.049867,33489052,2021年,石塚硝子,専01,燃料
4,石灰石使用量,6717.9,ｔ,440,2955876,2021年,石塚硝子,専01,工程
...,...,...,...,...,...,...,...,...,...
34,関西電力（ｾﾞﾛｶｰﾎﾞﾝﾒﾆｭｰを除く）,244837542,kWh,0.475,116297832,2013年,神戸製鋼所加古川製鉄所,専43,電気
0,都市ガス(13A),2380000,Nm3,0.049867,5340720,2021年,日本水産,専19,燃料
1,関西電力（ｾﾞﾛｶｰﾎﾞﾝﾒﾆｭｰを除く）,15306000,kWh,0.351,5372406,2021年,日本水産,専19,電気
2,都市ガス(13A),1834891,Nm3,0.049867,4117495,2013年,日本水産,専19,燃料


合計値を確認

In [ ]:
df_sum = df_detail.copy().groupby(['年次']).agg({'CO2排出量': 'sum'}).reset_index()
df_sum

,年次,CO2排出量
0,2013年,31227245318
1,2021年,26068284603


電気のCO2排出係数を0.250にした場合

## CO2排出明細の作成（電気0.250）

2030年に電気の排出係数が0.250になった場合の想定値

電気使用量は2021年とする

In [ ]:
def co2_2030(df_arg):

  df = df_arg.copy()
  
  # 2021年を抽出
  df = df[df['年次'] == '2021年']

  # 電気の排出係数を0.250に置換
  df['排出係数'] = df.apply(lambda x: x['排出係数'] if x['区分'] != '電気' else 0.250, 1)

  # 年次を2030年に置換
  df = df.replace({'年次': {'2021年': '2030年'}})

  return df

In [ ]:
df_detail_2030 = co2_2030(df_detail)
df_detail_2030

## データ整理

集計して整理

In [ ]:
# 2013年

def sum_co2(time,df_arg):
  
  df = df_arg.copy()
  
  # 単位をtに変換
  df['CO2排出量'] = df.apply(lambda x: x['CO2排出量'] / 1000, 1)
  df['CO2排出量'] = df['CO2排出量'].astype('int')
  
  # データ整形
  df = df.copy().groupby(['ID','会社名','年次','区分']).agg({'CO2排出量': 'sum'}).reset_index()
  df = df[df['年次'] == time]
  df = df.pivot(index=['ID','会社名'], columns='区分', values='CO2排出量').reset_index()
  df['合計'] = df.sum(axis=1)

    # CO2排出量を整数に変換
  # df = df.astype({'ID': str, '会社名': str, '区分': str, '工程': int, '区分': str, '区分': str, '区分': str, '区分': str})

  
  # 列名の変更
  columns = {'工程':f'{time}_工程【t】','燃料':f'{time}_燃料【t】','電気':f'{time}_電気【t】','合計':f'{time}_合計【t】'}
  df = df.rename(columns=columns)
  
  return df

In [ ]:
df_2013 = sum_co2('2013年',df_detail)
df_2021 = sum_co2('2021年',df_detail)

df = pd.merge(df_2013, df_2021, on=['ID','会社名'] , how='left')

df_2030 = sum_co2('2030年',df_detail_2030)

df = pd.merge(df, df_2030, on=['ID','会社名'] , how='left')

# 合計行を追加
df = df.append(df.sum(numeric_only=True), ignore_index=True)

df = df.fillna(0)

df